# Web of Science (WOS)

In [1]:
%load_ext autoreload

In [2]:
%autoreload

In [3]:
import pandas as pd
import re
import sys
import numpy as np
import time
from publications import *
from unidecode import unidecode


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)

In [4]:
def check_hash(df,hashseries,in_hash,min_match=10):
    ''' hashseries obtained from dataframe df, e.g
          hashseris=df.some_column.str.replace('\W+','').str.lower().map(unicode)
        within which in_hash will be searched for match at least min_match characters  
    '''
    comparision=True
    for si in reversed(range(0,len(in_hash)+1)):
        chk=df[hashseries.str.match(in_hash[:si])]
        if chk.shape[0]>0:
            return comparision,chk
            break    
        if si<min_match:
            comparision=False
            return comparision,pd.DataFrame()
        
            
def columns_add_prefix(df,prefix):
    return df.rename_axis( dict( (key,prefix+'_'+key) for key in df.columns.values) , axis=1)

def fill_NaN(df):
    '''Fill NaN entries with proper empty values
     Type  : dtype: Fill with
     string: "0"  : ''
     float : "float64" 
    '''
    for key in df.columns:
        if df[key].dtype=='O':
            df[key]=df[key].str.strip()
            df[key]=df[key].fillna('')
        elif df[key].dtype=='float64':
            df[key]=df[key].fillna(0.0)
    return df
def read_excel_fill_NaN(*args, **kwargs):
    '''Fill NaN entries with proper empty values
     Type  : dtype: Fill with
     string: "0"  : ''
     float : "float64" 
    '''
    df=pd.read_excel(*args, **kwargs)
    df=fill_NaN(df)
    return df
    
#To add to main publications object:
def add_sjr_info_from_issn(self,SJR,column_issn='SN',SJR_column_journal='SJR_Title',SJR_column_issn='SJR_Issn'):
    '''self is an publication object and SJR is the info for a journal in column SJR_Issn'''
    if not SJR_column_journal in self.articles.columns:
        sys.exit("Run first the the more exact and fast add_sjr_info")
            
    self.articles=fill_NaN(self.articles)
    kk=self.articles[self.articles[SJR_column_journal]=='']
    for issn in kk[column_issn].str.replace('-','').unique():
        mtch=SJR[SJR[SJR_column_issn].str.contains(issn)].reset_index(drop=True)
        if mtch.shape[0]>=1:
            moa=kk[ kk[column_issn].str.replace('-','')==issn ]
            if moa.shape[0]>=1:
                #DEBUG: more filters if 
                for key in SJR.columns.values:
                    self.articles.loc[moa.index.values,key]=mtch.ix[0][key]
                    
    return self

def add_sjr_info_from_journal(self,SJR,column_journal='SO',SJR_column_journal='SJR_Title'):
    '''self is an publication object and SJR is the info for a journal in column SJR_Issn'''
    if not SJR_column_journal in self.articles.columns:
        sys.exit("Run first the more exact and fast add_sjr_info")
        
    self.articles=fill_NaN(self.articles)
    self.articles_hash_SO=self.articles.SO.str.replace('\W+','').str.lower().str.strip().map(unidecode)
    SJR_hash_Source_title=SJR['SJR_Title'].str.replace('\W+','').str.lower().str.strip().map(unidecode)
    for in_hash in  SJR_hash_Source_title:
        check,moa=check_hash(self.articles,self.articles_hash_SO,in_hash,min_match=10)
        
        if moa.shape[0]:
            for i moa.index:
                for key in SJR.columns.values:
                    self.articles.loc[moa.index.values,key]=
                
    return self

def add_sjr_info(self,SJR,column_journal='SO',SJR_column_journal='SJR_Title'):
    '''self is an publication object and SJR is the info for a journal in column SJR_Title'''
    for joa in np.intersect1d( self.articles[column_journal].str.lower().str.strip().unique(),\
                                   SJR[SJR_column_journal].str.lower().str.strip().unique() ):
        moa=self.articles[ self.articles[column_journal].str.lower() == joa ]
        if moa.shape[0]:
            mtch=SJR[SJR[SJR_column_journal].str.lower().str.strip()==joa].reset_index(drop=True)
            if mtch.shape[0]>=1:
                #DEBUG: filter by ISSN if >1:
                for key in SJR.columns.values:
                    self.articles.loc[moa.index.values,key]=mtch.ix[0][key]
    
    return self
        


Design:
Creates new full data frame joining information from several databases. 
The initial data frame is in Web of Science format, with tge prefix `wos_` in the column names:
The new data is added as addtional columns in either an existing row identified by
* DOI
* Title and if necessary vol, first author surname, journal etc.
In the current implementation only DOI will be working.

In [5]:
sjr_oa=pd.read_excel('scimago/journals_scimago_open_access.xlsx')
sjr=pd.read_excel('scimago/journals_scimago_all.xlsx')

In [6]:
wos=articles(excel_file='wos_full_with_CR.xlsx')
wos.articles=fill_NaN(wos.articles)
wos.articles.shape

(6644, 55)

In [7]:
wos.articles[:1]

,AB,AF,AR,AU,BP,CR,DE,DT,EM,EP,FU,FX,GA,ID,IS,JI,LA,NR,PA,PD,PG,PI,PT,PU,PY,RP,SC,SN,SO,TC,TI,UT,VL,WC,DI,PM,RI,EI,OI,CL,CT,CY,SP,SU,BE,BN,HO,PN,SE,MA,SI,GP,CA,BA,BF
0,"Objectives: This work is intended to establish the prevalence of reverse smokers at the villages of Hato Nuevo, San Francisco and Cayo de Palma, Department of Sucre, Colombia, characterizing their...","Alvarez Gomez, Gloria J.\nAlvarez Martinez, Efrain\nJimenez Gomez, Raul\nMosquera Silva, Yolanda\nGaviria Nunez, Angela Maria\nGarces Agudelo, Adriana\nAlonso Duque, Alexander\nZabala Castano, Ale...",1111111172,"Gomez, GJA\nMartinez, EA\nGomez, RJ\nSilva, YM\nNunez, AMG\nAgudelo, AG\nDuque, AA\nCastano, AZ\nGonzalez, EE\nMillan, MI\nOssa, DR",E1,"Axell T, 1996, J ORAL PATHOL MED, V25, P49, DOI 10.1111/j.1600-0714.1996.tb00191.x\nBARIC JM, 1982, ORAL SURG ORAL MED O, V54, P424, DOI 10.1016/0030-4220(82)90389-9\nChang YC, 2001, J ORAL PATHOL...",oral cancer; oral premalignant lesions; reverse smoker; tobacco,Article,gloria@alvarez.nu,E8,CODI,"To the habitants of Hato Nuevo, San Francisco and Cayo de Palma, Sucre,\nColombia; to the field advisors for their help and attention in the\naccomplishment of this study and to the CODI by the pr...",352NW,SUBMUCOUS FIBROSIS; ANDHRA-PRADESH; SMOKING; LESIONS; INDIA;\nLEUKOPLAKIA; EPITHELIUM; CARCINOMA; CIGARETTE; NICOTINE,1,Med. Oral Patol. Oral Cir. Bucal,English,28,"CALLE DANIEL BALACIART N 4 PTA 17, VALENCIA, 46020, SPAIN",JAN,8,VALENCIA,J,MEDICINA ORAL S L,2008,"Gomez, GJA (reprint author), Univ Antioquia, Fac Dent, Calle 64 52-59, Medellin, Colombia.","Dentistry, Oral Surgery & Medicine",1698-4447,MEDICINA ORAL PATOLOGIA ORAL Y CIRUGIA BUCAL,4,"Reverse smokers's and changes in oral mucosa. Department of Sucre,\nColombia",WOS:000259504900001,13,"Dentistry, Oral Surgery & Medicine",,0,,,,,,,,,,,,,,,,,,,


## Include information from journals

In [8]:
start=time.time()
#prepare new columns
#prefix='SJR'
#SJR=sjr.rename_axis( dict( (key,prefix+'_'+key) for key in sjr.columns.values) , axis=1)
SJR=columns_add_prefix(sjr,'SJR')
#Add to exising dataframe
column_journal='SO'
self=wos    
self=add_sjr_info(self,SJR)    
print(time.time()-start)

58.863465547561646


In [9]:
self.articles=fill_NaN(self.articles)
ii=self.articles[self.articles.SJR_Title==''].index.values

In [10]:
start=time.time()
self=add_sjr_info_from_issn(self,SJR)
print(time.time()-start)

12.373026847839355


In [11]:
start=time.time()
self=add_sjr_info_from_journal(self,SJR)
print(time.time()-start)

27.581857681274414


In [12]:
res=wos.articles[wos.articles['SO'].str.lower().str.strip().str.contains('biomedica')]
res[['SO','SJR_Title']][:3]

,SO,SJR_Title
43,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud
44,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud
66,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud


## Includes papers from Scopus
* Add articles with doi into WOS: loop with the intersection between DOIs
* Creates a reduced pandas dataframe with not DOI and the DOIs not in intersection
* Add articles with title+author into reduced WOS: loop with the intersection between title+author
* append reduced pandas dataframe with title+author not in intersection


In [13]:
#sc=pd.read_csv('scopus/scopus2006-1977.csv').fillna('').drop('References',1)
#for fs in ['scopus/scopus2007.csv','scopus/scopus2010-2009-2008.csv',\
#          'scopus/scopus2012-2011.csv','scopus/scopus2014-2013.csv','scopus/scopus2016-2015.csv']:
#    print(fs)
#    sc=sc.append(pd.read_csv(fs,error_bad_lines=False))
#sc.to_excel('scp_full_with_Reference.xlsx',index=False)

In [14]:
scp=articles(excel_file='scp_full_with_Reference.xlsx')
scp.articles=scp.articles.rename_axis({'﻿Authors':'Authors'},1)
scp.articles=fill_NaN(scp.articles)

In [15]:
scp.articles[:1]

,Abbreviated Source Title,Abstract,Affiliations,Art. No.,Author Keywords,Authors with affiliations,CODEN,Chemicals/CAS,Cited by,Conference code,Conference date,Conference location,Conference name,Correspondence Address,DOI,Document Type,EID,Editors,Funding Details,ISBN,ISSN,Index Keywords,Issue,Language of Original Document,Link,Manufacturers,Molecular Sequence Numbers,Page count,Page end,Page start,PubMed ID,Publisher,References,Source,Source title,Sponsors,Title,Tradenames,Volume,Year,Authors
0,Gaceta Sanit.,"The Millennium Development Goals, and within these the Millennium Targets, constitute a working plan that strives to achieve basic goals within the field of health, eradicate poverty, and ensure c...","Área de Medicina Preventiva Y Salud Pública, Universidad de Alicante, Alicante, Spain; Facultad de Medicina, Universidad Autónoma de Yucatán, Yucatán, Mexico; Facultad Nacional de Salud Pública, U...",,Development; Millenium Goals; Political epidemiology; WHO,"González, D.G., Área de Medicina Preventiva Y Salud Pública, Universidad de Alicante, Alicante, Spain, Observatorio de Políticas Públicas Y Salud (OPPS), Spain, Área de Medicina Preventiva Y Salud...",,,5,0,,,,"González, D.G.; Área de Medicina Preventiva Y Salud Pública, Universidad de AlicanteSpain; email: Diana.Gil@ua.es",10.1157/13101091,Review,2-s2.0-34547700023,,,,02139111,epidemiology; motivation; politics; public health; review; social change; Epidemiology; Goals; Politics; Public Health; Social Change,SUPPL. 3,Spanish,https://www.scopus.com/inward/record.uri?eid=2-s2.0-34547700023&partnerID=40&md5=2155e0c6cdb235efd8a1953c4b49c370,,,0,65,61,17433202,,,Scopus,Gaceta Sanitaria,,The challenge to public health of the Millenium Development Goals: An approach from political epidemiology [El reto para la salud pública de los Objetivos de Desarrollo del Milenio: Un enfoque des...,,20,2006,"González D.G., Solís M.P., Cantero M.T.R., Moncada M.D.R.O., Giraldo Á.F., Stein A., Díaz C.Á.-D."


In [16]:
if wos.articles.shape[0]>0:
    wos.articles_hash=\
        wos.articles.TI.str.replace(r"[^a-zA-Z0-9 ]", " ").str.lower().str.strip().str.replace('\s+','').map(unidecode)
    scp.articles_hash=\
        scp.articles.Title.str.replace(r"[^a-zA-Z0-9 ]", " ").str.lower().str.strip().str.replace('\s+','').map(unidecode)

In [21]:
wos.articles_hash_SO=wos.articles.SO.str.replace('\W+','').str.lower().str.strip().map(unidecode)
scp.articles_hash_Source_title=scp.articles['Source title'].str.replace('\W+','').str.lower().str.strip().map(unidecode)

In [22]:
SCP=columns_add_prefix(scp.articles,'SCP')
SCP[:1]

,SCP_Abbreviated Source Title,SCP_Abstract,SCP_Affiliations,SCP_Art. No.,SCP_Author Keywords,SCP_Authors with affiliations,SCP_CODEN,SCP_Chemicals/CAS,SCP_Cited by,SCP_Conference code,SCP_Conference date,SCP_Conference location,SCP_Conference name,SCP_Correspondence Address,SCP_DOI,SCP_Document Type,SCP_EID,SCP_Editors,SCP_Funding Details,SCP_ISBN,SCP_ISSN,SCP_Index Keywords,SCP_Issue,SCP_Language of Original Document,SCP_Link,SCP_Manufacturers,SCP_Molecular Sequence Numbers,SCP_Page count,SCP_Page end,SCP_Page start,SCP_PubMed ID,SCP_Publisher,SCP_References,SCP_Source,SCP_Source title,SCP_Sponsors,SCP_Title,SCP_Tradenames,SCP_Volume,SCP_Year,SCP_Authors
0,Gaceta Sanit.,"The Millennium Development Goals, and within these the Millennium Targets, constitute a working plan that strives to achieve basic goals within the field of health, eradicate poverty, and ensure c...","Área de Medicina Preventiva Y Salud Pública, Universidad de Alicante, Alicante, Spain; Facultad de Medicina, Universidad Autónoma de Yucatán, Yucatán, Mexico; Facultad Nacional de Salud Pública, U...",,Development; Millenium Goals; Political epidemiology; WHO,"González, D.G., Área de Medicina Preventiva Y Salud Pública, Universidad de Alicante, Alicante, Spain, Observatorio de Políticas Públicas Y Salud (OPPS), Spain, Área de Medicina Preventiva Y Salud...",,,5,0,,,,"González, D.G.; Área de Medicina Preventiva Y Salud Pública, Universidad de AlicanteSpain; email: Diana.Gil@ua.es",10.1157/13101091,Review,2-s2.0-34547700023,,,,02139111,epidemiology; motivation; politics; public health; review; social change; Epidemiology; Goals; Politics; Public Health; Social Change,SUPPL. 3,Spanish,https://www.scopus.com/inward/record.uri?eid=2-s2.0-34547700023&partnerID=40&md5=2155e0c6cdb235efd8a1953c4b49c370,,,0,65,61,17433202,,,Scopus,Gaceta Sanitaria,,The challenge to public health of the Millenium Development Goals: An approach from political epidemiology [El reto para la salud pública de los Objetivos de Desarrollo del Milenio: Un enfoque des...,,20,2006,"González D.G., Solís M.P., Cantero M.T.R., Moncada M.D.R.O., Giraldo Á.F., Stein A., Díaz C.Á.-D."


In [23]:
newwos=pd.DataFrame()
for i in scp.articles.index:
    if i%500==0: print(i)
    #Try by title or DOI
    hash_match=True
    chk2=pd.DataFrame()
    hash_match,chk=check_hash(wos.articles,wos.articles_hash,scp.articles_hash[i])
        #validation: check for DOI or author
    if not hash_match and scp.articles.DOI.str.strip().ix[i]:
        chk=wos.articles[ wos.articles.DI==scp.articles.DOI.str.strip().ix[i] ]
        if chk.shape[0]:
            hash_match=True
    #Found good match by title or DOI:        
    if hash_match:
        #filter either with DOI or author
        if scp.articles.DOI.str.strip().ix[i]:
            chk2=chk[chk.DI==scp.articles.DOI.str.strip().ix[i]]
        
        if not chk2.shape[0]:
            chk2=chk[chk.AU.str.contains( unidecode( scp.articles.Authors.ix[i].split(' ')[0] ) ) ]
            if chk2.shape[0]:
                if chk2.shape[0]:  #take the first match: DEBUG: further filter woth journal name
                    if chk.DI.str.strip().ix[chk2.index.values[0]]:
                        scp.articles.loc[i,'DOI']=chk.DI.str.strip().ix[chk2.index.values[0]]
                    elif scp.articles.DOI.str.strip().ix[i]:
                        wos.articles.loc[chk2.index.values[0],'DI']=scp.articles.DOI.str.strip().ix[i]
                        
            else:
                hash_match=False
                
        #Finally check journal
        if hash_match:
            hash_match,chktmp=check_hash(wos.articles,wos.articles_hash_SO,scp.articles_hash_Source_title[i],min_match=9)
        
        
                    
                    
    #print(i,len(scp.articles_hash[i]),si,chk.shape[0],chk2.shape[0],hash_match)
    #if chk2.shape[0]: 
    #    print(wos.articles.SO[chk2.index.values[0]],':',scp.articles['Source title'][i])
    
    ##Actionschk.AU.str.contains(
    if hash_match:
        ##add columns to wos.articles
        for key in SCP.columns.values:
                wos.articles.loc[chk2.index.values[0],key]=SCP.ix[ i ][key]
        
    else:
        ##append new entry
        ##fill; wos.articles.SO; wos.articles.SN
        #break
        
        newwos=newwos.append( SCP.ix[ i ],ignore_index=True )
        newwos.loc[newwos.shape[0]-1,'SO']=scp.articles.ix[ i ]['Source title']
        newwos.loc[newwos.shape[0]-1,'SN']=re.sub('(^[0-9A-Z]{4})',r'\1-',\
                                                           scp.articles.ix[ i ]['ISSN'])
        #fill the other entries in proper format
    #if i>100:
    #    break

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500


In [24]:
wos.articles=wos.articles.append(newwos)
wos.articles=fill_NaN(wos.articles)

In [25]:
wos.articles.shape

(9938, 111)

In [26]:
kk=self.articles[self.articles.SO=='']

In [27]:
kk.shape

(0, 111)

In [28]:
res=wos.articles[wos.articles['SO'].str.lower().str.strip().str.contains('biomedica')]
res[['SO','SJR_Title','SCP_Source title']][:3]

,SO,SJR_Title,SCP_Source title
43,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud,Biomedica
44,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud,Biomedica
66,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud,Biomedica


## Tag open access papers based on WOS name database

## Tag open access papers based on Scimago SJR database based in SJR_Title
* Articles withoit SJR_Title and not in WOS Open Acces database cannot be tagged as Open Acces

In [43]:
start=time.time()
wos.articles['Open_Access']='No'
#sjr_open_acces_df=sjr_oa
for jrnl in np.intersect1d(sjr_oa.Title.str.lower().str.strip(),wos.articles.SO.str.lower().str.strip()):
    moa=wos.articles[ wos.articles.SO.str.lower().str.strip() == jrnl ]
    for j in moa.index:
        wos.articles.loc[j,'Open_Access']='Yes'
time.time()-start

23.128785610198975

In [44]:
wos.articles[wos.articles.Open_Access=='Yes'].shape

(4874, 112)

In [45]:
wos.articles[wos.articles.Open_Access=='Yes'].SO.str.lower().value_counts()

iatreia                                            456
revista colombiana de ciencias pecuarias           165
livestock research for rural development            85
revista mvz cordoba                                 78
vitae                                               73
revista colombiana de gastroenterologia             71
informacion tecnologica                             69
plos one                                            66
revista de salud publica                            64
colombia medica                                     64
revista colombiana de cardiologia                   50
revista colombiana de obstetricia y ginecologia     50
co-herencia                                         48
memorias do instituto oswaldo cruz                  48
revista colombiana de entomologia                   40
...
molecular ecology                                           1
nursing research                                            1
brain structure & function                     

Checks

Include (again) SJR information

In [48]:
start=time.time()
#prepare new columns
#prefix='SJR'
#SJR=sjr.rename_axis( dict( (key,prefix+'_'+key) for key in sjr.columns.values) , axis=1)
SJR=columns_add_prefix(sjr,'SJR')
#Add to exising dataframe
column_journal='SO'
self=wos    
self=add_sjr_info(self,SJR)    
print(time.time()-start)

118.40048456192017


In [49]:
self.articles=fill_NaN(self.articles)
ii=self.articles[self.articles.SJR_Title==''].index.values

In [50]:
start=time.time()
self=add_sjr_info_from_issn(self,SJR)
print(time.time()-start)

3.2758734226226807


In [51]:
start=time.time()
self=add_sjr_info_from_journal(self,SJR)
print(time.time()-start)

14.723922967910767


In [52]:
wos.articles['SJR_SJR Best Quartile'].value_counts()

Q1    3133
Q4    2450
Q2    2173
Q3    1781
-      287
       114
dtype: int64

In [53]:
wos.articles[wos.articles['SO'].str.lower().str.strip().str.contains('biomedica')][['SO','SJR_Title','SCP_Source title']]

,SO,SJR_Title,SCP_Source title
43,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud,Biomedica
44,BIOMEDICA,Biomedica : revista del Instituto Nacional de Salud,Biomedica
66,BIOMEDICA,Revista Chilena de Nutricion,Biomedica
85,BIOMEDICA,Revista Cubana de Estomatologia,Biomedica
86,BIOMEDICA,Livestock Research for Rural Development,Biomedica
144,BIOMEDICA,Ars Pharmaceutica,Biomedica
146,BIOMEDICA,Journal of Applied Physics,Biomedica
147,BIOMEDICA,Ingenieria Quimica,Biomedica
148,BIOMEDICA,Livestock Research for Rural Development,Biomedica
149,BIOMEDICA,Acta Farmaceutica Bonaerense,Biomedica


In [38]:
start=time.time()
#wos.articles=wos.articles
column_journal='SO'
wos.articles['Open_Access']='No'
#sjr_open_acces_df=sjr_oa
for jrnl in only_wos:
    moa=wos.articles[ wos.articles[column_journal].str.lower().str.strip() == jrnl.lower().strip() ]
    for j in moa.index:
        wos.articles.loc[j,'Open_Access']='Yes'
time.time()-start

0.23427200317382812

cms pas B2G-15-007
https://cds.cern.ch/record/2208044

MCFM

In [42]:
wos.articles[wos.articles.Open_Access=='Yes']['SO'].shape

(0,)

In [40]:
wos.articles[wos.articles.Open_Access=='Yes'].value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [33]:
js=['a','b','c']
jw=['a','c','d','e']
np.asarray([ j for j in jw if j not in js])

array(['d', 'e'], 
      dtype='<U1')

In [34]:
js=np.intersect1d(sjr_oa.Title.str.lower().str.strip(),wos.articles['SCP_Source title'].str.lower().str.strip())
jw=np.intersect1d(sjr_oa.Title.str.lower().str.strip(),wos.articles['SO'].str.lower().str.strip())
only_wos=np.asarray([ j for j in js if j not in jw])
only_wos.shape

(32,)

In [35]:
js.shape

(467,)

In [36]:
jw.shape

(456,)

In [ ]:
#    def add_sjr_info_from_journal(self,SJR,column_journal='SO',SJR_column_journal='SJR_Title'):
if 1==1:
    '''self is an publication object and SJR is the info for a journal in column SJR_Issn'''

    #if not SJR_column_journal in self.articles.columns:
        sys.exit("Run first the more exact and fast add_sjr_info")
        
    self.articles=fill_NaN(self.articles)
    self.articles_hash_SO=self.articles.SO.str.replace('\W+','').str.lower().str.strip().map(unidecode)
    SJR_hash_Source_title=SJR['SJR_Title'].str.replace('\W+','').str.lower().str.strip().map(unidecode)
    for in_hash in  SJR_hash_Source_title:
        check,moa=check_hash(self.articles,self.articles_hash_SO,in_hash,min_match=10)
        
        if moa.shape[0]:
            for i moa.index:
                for key in SJR.columns.values:
                    self.articles.loc[moa.index.values,key]=